In [1]:
import os
HOME = os.getcwd()
print(HOME)

/Users/zmhickey/Footballnotebooks/TeamTracking


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.9.16 torch-1.13.0 CPU
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 321.3/460.4 GB disk)


In [3]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [4]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [5]:

!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)
     

supervision.__version__: 0.1.0


In [6]:

from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [7]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections, 
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)
    
    tracker_ids = [None] * len(detections)
    
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids
     

In [8]:
WEIGHTS_PATH = f"{HOME}/runs/detect/train/weights/best.pt"

In [9]:
from ultralytics import YOLO

model = YOLO(WEIGHTS_PATH)
model.fuse()

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs


In [10]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [0, 3, 4, 5]

In [11]:
SOURCE_VIDEO_PATH=f"{HOME}/datasets/video3.mp4"

In [ ]:
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame)
detections = Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)
# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]
# annotate and display frame
frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (16, 16))

In [13]:
TARGET_VIDEO_PATH = f"{HOME}/v3Results.mp4"
VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=30, total_frames=432)

In [ ]:

from tqdm.notebook import tqdm


# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create LineCounter instance
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)


# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # format custom labels
        labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        sink.write_frame(frame)
     

In [ ]:
import math
from tqdm.notebook import tqdm
import cv2


# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create LineCounter instance
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=1, text_thickness=1, text_scale=.5)


# open target video file
TeamInPos = 0
Team1TotalPos = 0
Team2TotalPos = 0
TotalTime = 0

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model.predict(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        #detections.filter(mask=mask, inplace=True) #this gets rid of some of the things ie ball go away
        
        # format custom labels
        labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        
        # check ball pos
        
        ballFound = False 
        for detection in detections:
            #print(detection[0])
            x  = (detection[0][0] + detection[0][1]) / 2
            y = (detection[0][2] + detection[0][3]) / 2
            ballPos = np.array( [x,y])
            
            #print(ballPos)
            # ball
            if(detection[2] == 0):
                #print('ball found')
                ballPos = np.array([ (detection[0][0] + detection[0][1]) / 2, (detection[0][2] + detection[0][3]) / 2 ])
                ballFound = True
                #print(ballPos)
            # team 1
            players = []
            if(detection[2] == 4):
                x  = (detection[0][0] + detection[0][1]) / 2
                y = (detection[0][2] + detection[0][3]) / 2
                players.append([4,x,y])
            # team 2
            if(detection[2] == 5):
                x  = (detection[0][0] + detection[0][1]) / 2
                y = (detection[0][2] + detection[0][3]) / 2
                players.append([5,x,y])           
            players = np.array(players)
            
            #check team in pos
            Team1Pos = False
            Team2Pos = False
            if(ballFound):
                for player in players:
                    dist = math.sqrt((ballPos[0] - player[1])**2 + (ballPos[1] - player[2])**2)
                    if(dist < 30):
                        if(player[0] == 4):
                            Team1Pos = True
                        else: 
                            Team2Pos = True
            # check who has the ball
            if(Team1Pos and Team2Pos):
                TeamInPos = 0
            elif(Team1Pos):
                TeamInPos = 1
            elif(Team2Pos):
                TeamInPos = 2
            # increment team in pos and total time
            if(TeamInPos == 1):
                Team1TotalPos += 1
            if(TeamInPos == 2):
                Team2TotalPos += 1
            TotalTime +=1

            #create bar with pos 
            color = (255, 0, 0) 
            x, y, h = 100, 100, 20
            T1curAmount = (Team1TotalPos/TotalTime)
            inCostCuramount = (TotalTime - Team1TotalPos - Team2TotalPos)/TotalTime
            T2curamount = (TotalTime - Team1TotalPos - Team2TotalPos)/TotalTime
            x1 = 100 + int(600 *T1curAmount)
            x2 = x1 + int(600 * inCostCuramount)
            #team1
            color = (255, 0, 0) 
            cv2.rectangle(frame, (x, 100), (x1, 120), color, thickness=-1)

            #incontest
            color = (0, 225, 0) 
            cv2.rectangle(frame, (x1, 100), (x2, 120), color, thickness=-1)

            #team2
            color = (0, 0, 225) 
            cv2.rectangle(frame, (x2, 100), (600, 120), color, thickness=-1)


        sink.write_frame(frame)
#prints pos of teams
print('team1 pos: ' , 100 * (Team1TotalPos/TotalTime) )
print('team2 pos: ' , 100 * (Team2TotalPos/TotalTime))
print('inContest: ', 100 * ((TotalTime - Team1TotalPos - Team2TotalPos)/TotalTime) )
print('total time: ', TotalTime )